<a href="https://colab.research.google.com/github/Pinlinzz/Analisis-Dataset-Employee-Salary-Analysis-/blob/main/Analisis_Dataset_Employee_Salary_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INISIASI DAN IMPORTING DATASET


Let's check the structure and summary of the R data frame, similar to `df.info()` and `df.describe()` in Python.

In [15]:
# Load the dataset into an R data frame
salary_df <- read.csv('/content/dataset/Salary_Dataset.csv')

# Display the first 5 rows of the data frame
summary(salary_df)

  Employee_ID         Name                Age           Gender         
 Min.   :   1.0   Length:1200        Min.   :20.00   Length:1200       
 1st Qu.: 300.8   Class :character   1st Qu.:30.00   Class :character  
 Median : 600.5   Mode  :character   Median :41.00   Mode  :character  
 Mean   : 600.5                      Mean   :39.99                     
 3rd Qu.: 900.2                      3rd Qu.:50.00                     
 Max.   :1200.0                      Max.   :59.00                     
   Country              City            Education          Job_Title        
 Length:1200        Length:1200        Length:1200        Length:1200       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                  

# Data Cleaning & Preprocessing
